# Calculation of the the gradient in cylindrical coordinates using contravariant vector composents

The strategy is to

* Calculate the vector in cartesian coordinates
* Transform them to a contravariantly cylindrical vector (transform the elements)

This way is chosen as the differential geometry package currently misses out the differential operations.
We will use a simple stupid approach, as that is what we need for these simple tasks

In [ ]:
from collections import OrderedDict
from sympy import init_printing
from sympy import symbols, simplify, solve
from sympy import Eq
from sympy import exp, sqrt, atan, cos, sin
import re
from IPython.display import display

init_printing()

Define symbols

In [ ]:
mu = symbols('mu', positive = True)
rho, rho0, z, z0 = symbols('rho, rho0, z, z0', positive = True)
theta, theta0 = symbols('theta, theta0', real = True)
x, y, z = symbols('x, y, z', real = True)
x0, y0, z0 = symbols('x0, y0, z0', real = True)
Ax, Ay, Az = symbols('A^x, A^y, A^z', real = True)
cart_symb = [x, y, z]
A_symb = [Ax, Ay, Az]

Define our maps

In [ ]:
cylToCart = OrderedDict([(rho,sqrt(x**2+y**2)),
                         (theta, atan(y/x)),
                         (z, 1*z)])

for coord in cylToCart.keys():
    display(Eq(coord,cylToCart[coord]))

Unfortunately, the next cell gives unsatisfactory results (gives two solution where it should have been only one), so the inverse map needs to be inserted manually

In [ ]:
# As z = z => True, we need to cast the coordinate to symbols(str())
eqs = tuple([Eq(symbols(str(coord)), cylToCart[coord]) for coord in cylToCart.keys()])
sol = solve(eqs, x, y, z)
display(sol)

Manually insert the inverse transform

In [ ]:
print("Normal transform")
cartToCyl = OrderedDict([(x, rho*cos(theta)),
                         (y, rho*sin(theta)),
                         (z, z)])

for coord in cartToCyl.keys():
    display(Eq(coord,cartToCyl[coord]))

print("Transform for the 0 components")
cartToCyl0 = OrderedDict([(x0, rho0*cos(theta0)),
                          (y0, rho0*sin(theta0)),
                          (z0, z0)])

for coord in cartToCyl0.keys():
    display(Eq(coord,cartToCyl0[coord]))

We will now derive the recipie for the vector in the new coordinate system. See example 4 in https://github.com/loeiten/differential_geometry/tree/master/notes

In [ ]:
cylTemplateCart = OrderedDict.fromkeys(cylToCart)
print("Contravariant components written in cartesian coordinates")
for coord in cylTemplateCart.keys():
    cylTemplateCart[coord]  = 0
    for cart, A in zip(cart_symb, A_symb):
        cylTemplateCart[coord]  += A*cylToCart[coord].diff(cart)
    display(Eq(symbols('v^'+str(coord)),cylTemplateCart[coord]))

cylTemplateCyl = cylTemplateCart.copy()
print("\n Contravariant components written in cylindrical coordinates")
replace = [(coord, cartToCyl[coord]) for coord in cartToCyl.keys()]
for coord in cylTemplateCyl.keys():
    cylTemplateCyl[coord] = simplify(cylTemplateCyl[coord].subs(replace))
    display(Eq(symbols('v^'+str(coord)),cylTemplateCyl[coord]))

Define our function to take the gradient of (in cartesian coordinates)

In [ ]:
f = exp(-((x - x0)**2 + (y - y0)**2 + (z-z0)**2)/mu**2)
display(Eq(symbols('f'), f))

We now calculate the divergence in cartesian coordinates

In [ ]:
cartVec = OrderedDict([(x, simplify(f.diff(x))),
                       (y, simplify(f.diff(y))),
                       (z, simplify(f.diff(z)))])

for coord in cartVec.keys():
    display(Eq(symbols('A^' + str(coord)),cartVec[coord]))

Now we write this in cylindrical coordinates

In [ ]:
replace =  [(coord, cartToCyl[coord]) for coord in cartToCyl.keys()]
replace0 = [(coord, cartToCyl0[coord]) for coord in cartToCyl0.keys()]
cartVecCyl = cartVec.copy()
for coord in cartVec.keys():
    cartVecCyl[coord] = cartVecCyl[coord].subs(replace)
    cartVecCyl[coord] = simplify(cartVecCyl[coord].subs(replace0))
    display(Eq(symbols('A^'+str(coord)),cartVecCyl[coord]))

We now insert $A^i$ in the $v^j$ components

In [ ]:
# Appendable list
replace = []
coords = cartVecCyl.keys()
for A, coord in zip(A_symb, coords):
    replace.append((A, cartVecCyl[coord]))

replace = tuple(replace)
cylVec = cylTemplateCyl.copy()
for coord in cylVec.keys():
    cylVec[coord] = simplify(cylVec[coord].subs(replace))
    display(Eq(symbols('v^'+str(coord)),cylVec[coord]))

We would like to be able to copy paste the above directly into BOUT++

In [ ]:
def BOUTstring(string, cylindrical=False):
    outstring = string.replace('**', '^')
    
    replacements = [('mu', 'cst:mu'),
                    ('rho', 'geom:xl'),
                    ('theta', 'z'),
                    ('x', 'geom:xl'),
                    ('y', 'geom:yl'),
                    ('rho0', 'cst:rho0'),
                    ('theta0', 'cst:theta0'),
                    ('x0', 'cst:x0'),
                    ('y0', 'cst:y0'),
                    ('z0', 'cst:z0'),
                   ]
    
    if cylindrical:
        # Insert this in the front, so that we don't first replace theta to z, then z to geom:yl
        replacements.insert(0, ('z', 'geom:yl'))
    
    for replacement in replacements:
        outstring = re.sub(r'\b'+replacement[0]+r'\b', replacement[1], outstring)
        
    return outstring

Print the BOUT++ strings

In [ ]:
for coord in cylVec.keys():
    print("{0}_contravariant = {1}".format(coord, BOUTstring(str(cylVec[coord]), cylindrical=True)) + "\n"*2)
    
for coord in cartVec.keys():
    print("{0} = {1}".format(coord, BOUTstring(str(cartVec[coord]))) + "\n"*2)